# 两层FC做分类

使用两层全连接做分类，输入[-1,28*28],FC1有1024个neurons,FC2有10个neurons.

In [1]:
from __future__ import print_function
from __future__ import division #精确除法 使/是精确的除法
from __future__ import absolute_import#绝对引入

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf
sess = tf.Session()

### 1.导入数据

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/MNIST_data', one_hot=False, source_url='http://yann.lecun.com/exdb/mnist/')

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print('training data shape', mnist.train.images.shape)
print('training label shape', mnist.train.labels.shape)

training data shape (55000, 784)
training label shape (55000,)


### 2.构建网络

In [4]:
#权值初始化
def weight_variable(shape):
    #用正态分布初始化权值
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    #relu作为激活函数，一般偏置较小
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
#input layer
X_input = tf.placeholder(tf.float32, [None, 784])
y_input = tf.placeholder(tf.int64, [None])

#FC1
W_fc1 = weight_variable([784, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(X_input, W_fc1) + b_fc1)

#FC2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
logits = tf.matmul(h_fc1, W_fc2) + b_fc2

print(logits)

Tensor("add_1:0", shape=(?, 10), dtype=float32)


### 3.训练&评估

In [6]:
#1.损失函数：cross_entropy
# 如果label 是one-hot的话损失函数要更换 参考https://blog.csdn.net/tz_zs/article/details/76086457
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_input, logits=logits))
# 2.优化函数：AdamOptimizer,优化速度要比GradientOptimizer快很多
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
#3.预测结果评估
correct_prediction = tf.equal(tf.argmax(logits, 1), y_input)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#开始运行
sess.run(tf.global_variables_initializer())
for i in range(5000):
    X_batch, y_batch = mnist.train.next_batch(batch_size=100)
    cost, acc, _ = sess.run([cross_entropy, accuracy, train_step], feed_dict={X_input:X_batch, y_input:y_batch})
    if (i + 1) % 500 == 0:
        test_cost, test_acc = sess.run([cross_entropy, accuracy], feed_dict={X_input:mnist.test.images, y_input:mnist.test.labels})
        print("step {}, train cost = {:.6f}, acc={:.6f};test cost={:.6f}, acc={:.6f}".format(i+1, cost, acc, test_cost, test_acc))

step 500, train cost = 0.224238, acc=0.950000;test cost=0.130887, acc=0.959500
step 1000, train cost = 0.146907, acc=0.970000;test cost=0.095745, acc=0.969600
step 1500, train cost = 0.045959, acc=0.980000;test cost=0.077362, acc=0.976000
step 2000, train cost = 0.040121, acc=0.990000;test cost=0.071335, acc=0.977500
step 2500, train cost = 0.038331, acc=0.990000;test cost=0.069818, acc=0.977700
step 3000, train cost = 0.011953, acc=1.000000;test cost=0.076723, acc=0.978400
step 3500, train cost = 0.018935, acc=1.000000;test cost=0.062385, acc=0.981200
step 4000, train cost = 0.027220, acc=0.990000;test cost=0.079847, acc=0.978700
step 4500, train cost = 0.002668, acc=1.000000;test cost=0.068271, acc=0.981400
step 5000, train cost = 0.001139, acc=1.000000;test cost=0.065974, acc=0.982500
